<a href="https://colab.research.google.com/github/Swleiii/Coursework/blob/main/Tablet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Preparation Codes

In [ ]:
# -- Install and setup Oracle Instant Client --
! wget -O oracleinstantclient.zip https://download.oracle.com/otn_software/linux/instantclient/19600/instantclient-basic-linux.x64-19.6.0.0.0dbru.zip
! unzip -oq oracleinstantclient.zip
! mkdir -p /opt/oracle
! mv instantclient_19_6 /opt/oracle/
! sh -c "echo /opt/oracle/instantclient_19_6 > /etc/ld.so.conf.d/oracle-instantclient.conf"
! ldconfig


# -- Include cx_Oracle package --
! pip install cx_Oracle

In [ ]:
import cx_Oracle

HOST_NAME = "imz409.ust.hk"
PORT_NUMBER = "1521"
SERVICE_NAME = "imz409"
USERNAME = "LA303"
PASSWORD = "HK12"

dsn_tns = cx_Oracle.makedsn(HOST_NAME, PORT_NUMBER, service_name=SERVICE_NAME)
conn = cx_Oracle.connect(user=USERNAME, password=PASSWORD, dsn=dsn_tns)

c = conn.cursor()

In [ ]:
from google.colab.widgets import TabBar
import ipywidgets as widgets
import time
import pandas as pd
from ipywidgets import interact
from ipywidgets import Text
from ipywidgets import DatePicker
from ipywidgets import IntText
from ipywidgets import Label
from ipywidgets import GridspecLayout
from ipywidgets import Layout
from datetime import date
import random

In [ ]:
###toolbar
tb_all = TabBar(['Manager Login', 'Function Selection','Assign Table','Reset Tablet','Customer Login','Menu','Order Cart','Order History','Category','Popular','Message Box'])

assignedTable = None
# a simple function for jumping between tabs
def showPage(tabName):
  with tb_all.output_to(tabName):
    display()

orderCart = {}
transID = None

def getOrderCart():
    global orderCart
    return orderCart


#returns whether the cart was updated
def update_cart(foodId, qty):
  ordercart = getOrderCart()
  if foodId not in ordercart and len(ordercart)>=4:
    return False
  elif qty == 0:
    del orderCart[foodId]
    return True
  else:
    orderCart[foodId]=qty
    return True

def resetCart():
  global orderCart
  orderCart = {}

def show_message(currentPage, message):
  global prevPage
  prevPage = currentPage
  messageBox_label.value=message
  showPage('Message Box')

def ordHisInd(pagenum,ls):
    mpp=4
    maximum = pagenum*mpp
    values = int()
    if len(ls)<maximum:
      maxrange = mpp - (maximum-len(ls))
      values = ls[-maxrange:]
    else:
      minr = (pagenum-1)*4
      maxr = (pagenum-1)*4+4
      values=ls[minr:maxr]
    return values

ohMaxDisplayPage = 0
def getOrderHistory():
  global orderHis, orderHisSorted, ohMaxDisplayPage
  orderHis = {}
  c.execute(f'SELECT co_id,co_submission_time,f_name,co_qty,f_price,co_status FROM cooking_order LEFT JOIN menu ON cooking_order.menu_f_id=menu.f_id WHERE transaction_tr_id = {transID}')
  for a in c:
    orderHis[a[0]]={'co_submission_time':a[1],'f_name':a[2],'co_qty':a[3],'f_price':a[4],'co_status':a[5]}
  orderHisSorted = dict(sorted(orderHis.items(), key=lambda item: item[1]['co_submission_time']))
  if len(orderHisSorted)%4!=0:
    ohMaxDisplayPage = (len(orderHisSorted)//4)+1
  else:
    ohMaxDisplayPage = len(orderHisSorted)//4
  return ordHisInd(orderHis_currPage,list(orderHisSorted.keys()))


## Message box
with tb_all.output_to('Message Box'):
  messageBox_label = Label("")
  btnMsgBoxOK = widgets.Button(description="OK")

  def on_btnMsgBoxOK_button_clicked(b):
    showPage(prevPage)
  btnMsgBoxOK.on_click(on_btnMsgBoxOK_button_clicked)

  intro_row1HBox = widgets.HBox([messageBox_label])
  intro_row2HBox = widgets.HBox([btnMsgBoxOK])

  display(widgets.VBox([intro_row1HBox, intro_row2HBox]))

## function selection
with tb_all.output_to('Function Selection'):
  assign_table_button = widgets.Button(description="Assign Table")
  reset_tablet_button = widgets.Button(description="Reset Tablet")
  fsToMenuBtn = widgets.Button(description='To Order Menu')

  ##BUG EDITING CHANGES tableDropdown.value -> assignedTable
  def fsToMenuBtnClicked(b):
    global prevPage
    if assignedTable == None:
      messageBox_label.value='Tablet is currently not assigned to any tables'
      prevPage = 'Function Selection'
      showPage('Message Box')
    else:
      showPage('Menu')
  fsToMenuBtn.on_click(fsToMenuBtnClicked)
  def assign_table_button_clicked(b):
    global prevPage
    prevPage = 'Function Selection'
    if assignedTable != None:
      messageBox_label.value='Tablet is currently assigned to table '+assignedTable+'. Please reset the tablet before assigning this tablet to another table.'
      showPage('Message Box')
    else:
      tableDropdownls = []
      c.execute("SELECT TAB_ID FROM TABLES WHERE TAB_STATUS = 'Available'")
      for row in c:
        tableDropdownls.append(row[0])
      tabledataDt = {}
      for a in tableDropdownls:
        if int(a) <4:
          tabledataDt[a] = 'Table {}, Max Capacity = 2'.format(a)
        else:
          tabledataDt[a] = 'Table {}, Max Capacity = 4'.format(a)
      tableDropdown = widgets.Dropdown(options=tabledataDt.values(), value=None)
      if len(tableDropdownls)>0:
        showPage('Assign Table')
      else:
        messageBox_label.value='No available tables'
        showPage('Message Box')
  assign_table_button.on_click(assign_table_button_clicked)

  def reset_table_button_clicked(b):
    global prevPage
    prevPage = 'Function Selection'
    if assignedTable != None:
      c.execute(f"SELECT TR_PAYMETHOD FROM TRANSACTION WHERE TR_ID = '{transID}'")
      tr_status = ''
      for a in c:
        tr_status+=a[0]
      if tr_status!= 'In Transaction':
        showPage('Reset Tablet')
      else:
        messageBox_label.value='Please settle bill on cashier terminal before resetting table'
        showPage('Message Box')
    else:
      messageBox_label.value = 'Tablet is currently not assigned to any tables'
      showPage('Message Box')

  reset_tablet_button.on_click(reset_table_button_clicked)

  function_selection_line1HBox = widgets.HBox([assign_table_button, reset_tablet_button])
  function_selection_line2HBox = widgets.HBox([fsToMenuBtn])

  display(widgets.VBox([function_selection_line1HBox,function_selection_line2HBox]))

## assign table
with tb_all.output_to('Assign Table'):
  hcTitle = Label("Headcount")
  hcDropdown = widgets.Dropdown(options=["1","2","3","4"], value=None)

  tableTitle = Label("Table")
  tableDropdownls = []
  c.execute("SELECT TAB_ID FROM TABLES WHERE TAB_STATUS = 'Available'")
  for row in c:
    tableDropdownls.append(row[0])
  tabledataDt = {}
  for a in tableDropdownls:
    if int(a) <4:
      tabledataDt[a] = 'Table {}, Max Capacity = 2'.format(a)
    else:
      tabledataDt[a] = 'Table {}, Max Capacity = 4'.format(a)
  tableDropdown = widgets.Dropdown(options=tabledataDt.values(), value=None)


  confirmTableBtn = widgets.Button(description="Confirm Selection")
  assignTableBackBtn = widgets.Button(description='Back')

  def assignTableBackBtnClicked(b):
    showPage('Function Selection')

  def confirmTableBtnClick(b):
    global prevPage
    global assignment
    global ATpage
    global fromPage
    global selectedTable
    selectedTable = tableDropdown.value[6]
    prevPage = 'Assign Table'
    if hcDropdown.value==None or tableDropdown.value==None:
      messageBox_label.value = 'Please select a value'
      showPage('Message Box')
    else:
      selected = c.execute('SELECT TAB_CAPACITY FROM TABLES WHERE TAB_ID={}'.format(selectedTable))
      selectedTableCap = ''

      for a in selected:
        selectedTableCap+=a[0]
      if int(hcDropdown.value) > int(selectedTableCap):
        messageBox_label.value='Exceeds Table Maximum Capacity'
        showPage('Message Box')
      else:
        ATpage = True
        assignment = False
        fromPage = 'Assign Table'
        showPage('Manager Login')

  assignTableBackBtn.on_click(assignTableBackBtnClicked)
  confirmTableBtn.on_click(confirmTableBtnClick)

  assign_table_labelVBox = widgets.VBox([hcTitle,tableTitle])
  assign_table_dropdown = widgets.VBox([hcDropdown, tableDropdown])
  assign_table_line1HBox = widgets.HBox([assign_table_labelVBox, assign_table_dropdown])
  assign_table_line2HBox = widgets.HBox([confirmTableBtn, assignTableBackBtn])


  display(widgets.VBox([assign_table_line1HBox,assign_table_line2HBox]))

#customer login
with tb_all.output_to('Customer Login'):
  cusPhone = Label("Phone Number")
  cusPhone_textbox = Text(value='', placeholder='Phone Number', disabled=False)

  confirmNumBtn = widgets.Button(description="Confirm")
  skipBtn = widgets.Button(description="Skip")

  def confirmBtnClick(b):
    if cusPhone_textbox.value == "":
      global prevPage
      prevPage = 'Customer Login'
      messageBox_label.value = 'Please input a valid value'
      showPage('Message Box')
    else:
      existingTrID = c.execute("SELECT TR_ID FROM TRANSACTION")
      existingTrIDls = [a[0] for a in existingTrID]
      global transID, allfName, allfDesc,allfCat, allfPrice, allfAvail, allfID, allfInv, categoryItems, allFoodItemsDt
      transIDgen = [str(random.randint(0,9)) for a in range(8)]
      transID = ''.join(transIDgen)
      while transID not in existingTrIDls:
        existingTrIDls.append(transID)
      else:
        transIDgen = [str(random.randint(0,9)) for a in range(8)]
        transID = ''.join(transIDgen)
        existingTrIDls.append(transID)
      c.execute("""INSERT INTO TRANSACTION VALUES ('{}',SYSDATE,'','{}',{},{},{},'{}','{}')""".format(transID,"In Transaction",hcDropdown.value,"0",'0',cusPhone_textbox.value,selectedTable))
      c.execute('commit')
          #getting items
      allfName, allfDesc,allfCat, allfPrice, allfAvail, allfID, allfInv = [],[],[],[],[],[],[]
      c.execute("SELECT f_name,f_description,f_category,f_price,f_availability,f_id,f_inventory FROM MENU WHERE f_availability = 'Available'")
      for a in c:
        allfName.append(a[0])
        allfDesc.append(a[1])
        allfCat.append(a[2])
        allfPrice.append(a[3])
        allfAvail.append(a[4])
        allfID.append(a[5])
        allfInv.append(a[6])
      allFoodItemsDt={}
      for a in range(len(allfID)):
        allFoodItemsDt[allfID[a]] = [allfID[a],allfName[a],allfDesc[a],allfCat[a],allfPrice[a],allfAvail[a],allfInv[a],0]
      c.execute("SELECT sum(co_qty), menu_f_id FROM cooking_order GROUP BY menu_f_id")
      salesVolume={a[1]:a[0] for a in c}
      for k,v in allFoodItemsDt.items():
        if k in salesVolume:
          v.append(salesVolume[k])
        else:
          v.append(0)
      cusPhone_textbox.value = ""
      showPage('Menu')

  def skipBtnClick(b):
    existingTrID = c.execute("SELECT TR_ID FROM TRANSACTION")
    existingTrIDls = [a[0] for a in existingTrID]
    global transID
    transIDgen = [str(random.randint(0,9)) for a in range(8)]
    transID = ''.join(transIDgen)
    while transID not in existingTrIDls:
      existingTrIDls.append(transID)
    else:
      transIDgen = [str(random.randint(0,9)) for a in range(8)]
      transID = ''.join(transIDgen)
      existingTrIDls.append(transID)
    c.execute("""INSERT INTO TRANSACTION VALUES ('{}',SYSDATE,'','{}',{},{},{},'','{}')""".format(transID,"In Transaction",hcDropdown.value,"0",'0',selectedTable))
    c.execute('commit')
        #getting items
    global allfName, allfDesc,allfCat, allfPrice, allfAvail, allfID, allfInv, categoryItems, allFoodItemsDt
    allfName, allfDesc,allfCat, allfPrice, allfAvail, allfID, allfInv = [],[],[],[],[],[],[]
    c.execute("SELECT f_name,f_description,f_category,f_price,f_availability,f_id,f_inventory FROM MENU WHERE f_availability = 'Available'")
    for a in c:
      allfName.append(a[0])
      allfDesc.append(a[1])
      allfCat.append(a[2])
      allfPrice.append(a[3])
      allfAvail.append(a[4])
      allfID.append(a[5])
      allfInv.append(a[6])
    allFoodItemsDt={}
    for a in range(len(allfID)):
      allFoodItemsDt[allfID[a]] = [allfID[a],allfName[a],allfDesc[a],allfCat[a],allfPrice[a],allfAvail[a],allfInv[a],0]
    c.execute("SELECT sum(co_qty), menu_f_id FROM cooking_order GROUP BY menu_f_id")
    salesVolume={a[1]:a[0] for a in c}
    for k,v in allFoodItemsDt.items():
      if k in salesVolume:
        v.append(salesVolume[k])
      else:
        v.append(0)
    showPage('Menu')

  confirmNumBtn.on_click(confirmBtnClick)
  skipBtn.on_click(skipBtnClick)

  cus_login_labelVBox = widgets.VBox([cusPhone,cusPhone_textbox])
  cus_login_line1HBox = widgets.HBox([cus_login_labelVBox])
  cus_login_line2HBox = widgets.HBox([confirmNumBtn, skipBtn])


  display(widgets.VBox([cus_login_line1HBox,cus_login_line2HBox]))


#menu
with tb_all.output_to('Menu'):
  allFoodItemsDt={}
  manager_login_button = widgets.Button(description="Manager Login")
  assignedTableNo = Label('Table: Not assigned')
  orderHistoryBtn = widgets.Button(description="Order History")
  orderCartBtn = widgets.Button(description="Order Cart")
  popularBtn = widgets.Button(description="View Top 5 Items")
  categoryBtn = widgets.Button(description="View by Categories")
  menu_spacer = Label('----------------------------------------------------------------------------------------------------------')
  def popularBtnClick(b):
    global prevPage
    prevPage = 'Menu'
    displayPopItems(get_top_5_popular_items())
    showPage('Popular')
  def categoryBtnClick(b):
    global prevPage
    prevPage = 'Menu'
    emptyBoard()
    showPage('Category')
  def orderHistoryBtnClick(b):
    global prevPage
    prevPage = 'Menu'
    ordHis_display(getOrderHistory())
    showPage('Order History')
  def orderCartBtnClick(b):
    global prevPage
    prevPage = 'Menu'
    displayOrderCart(getOrderCart())
    showPage('Order Cart')
  def manager_login_button_click(b):
    showPage('Manager Login')
    global fromPage
    global prevPage
    prevPage='Menu'
    fromPage='Menu'
  manager_login_button.on_click(manager_login_button_click)
  orderHistoryBtn.on_click(orderHistoryBtnClick)
  orderCartBtn.on_click(orderCartBtnClick)
  popularBtn.on_click(popularBtnClick)
  categoryBtn.on_click(categoryBtnClick)

  menu_line1HBox = widgets.HBox([manager_login_button, assignedTableNo])
  menu_line2HBox = widgets.HBox([orderCartBtn, orderHistoryBtn])
  menu_line3HBox = widgets.HBox([menu_spacer])
  menu_line4HBox = widgets.HBox([popularBtn])
  menu_line5HBox = widgets.HBox([categoryBtn])
  display(widgets.VBox([menu_line1HBox,menu_line2HBox,menu_line3HBox,menu_line4HBox,menu_line5HBox]))

#reset tablet
with tb_all.output_to('Reset Tablet'):
  resetTabNo = Label("This tablet is assigned to ")
  resetBtn = widgets.Button(description="Reset")
  resetMenuBtn = widgets.Button(description="Back to Menu")
  def resetBtn_click(b):
    global selectedTable
    assignedTableNo.value = 'Table: Not Assigned'
    c.execute("UPDATE TABLES SET TAB_STATUS = 'Available' WHERE TAB_ID = {}".format(selectedTable))
    c.execute("commit")
    orderHis = {}
    messageBox_label.value = 'Table '+str(selectedTable) + ' has been reset'
    global assignedTable
    assignedTable = None
    tableDropdown.value = None
    selectedTable = None
    global prevPage
    prevPage = 'Function Selection'
    showPage('Message Box')
  def resetMenuBtnClicked(b):
    showPage('Menu')
  resetMenuBtn.on_click(resetMenuBtnClicked)
  resetBtn.on_click(resetBtn_click)

  resetTab_line1HBox = widgets.HBox([resetTabNo])
  resetTab_line2HBox = widgets.HBox([resetBtn,resetMenuBtn])
  display(widgets.VBox([resetTab_line1HBox,resetTab_line2HBox]))

## Manager login page

with tb_all.output_to('Manager Login'):
  man_login_id_label = Label ("Manager ID: ")
  man_login_id_textbox = Text(value='', placeholder='Manager ID', disabled=False)

  man_pw_label = Label("Manager PW: ")
  man_pw_textbox = Text(value='', placeholder='Manager PW', disabled=False)

  man_back_button = widgets.Button(description = "Back")
  login_button = widgets.Button(description="Login")

  def man_back_button_clicked(b):
    showPage(fromPage)

  def on_login_button_clicked(b):
    loginID = man_login_id_textbox.value.strip()
    loginPW = man_pw_textbox.value.strip()
    data = c.execute("SELECT MAN_ID,MAN_PW FROM MANAGER WHERE MAN_ID = '{}' AND MAN_PW = '{}'".format(loginID,loginPW))
    ls_data = [row for row in data]
    global prevPage
    global ATpage
    global assignment
    if len(ls_data)>0:
      if ATpage == True and assignment == False:
        ATpage = False
        assignment = True
        global assignedTableNo
        global assignedTable
        global resetTabNo
        assignedTable=selectedTable
        assignedTableNo.value='Table: '+str(selectedTable)
        c.execute("UPDATE TABLES SET TAB_STATUS='Unavailable' WHERE TAB_ID={}".format(selectedTable))
        c.execute("commit")
        resetTabNo.value = 'This tablet is assigned to table '+str(selectedTable)
        showPage('Customer Login')
      else:
        showPage('Function Selection')
    else:
      messageBox_label.value='Incorrect Password'
      prevPage = 'Manager Login'
      showPage('Message Box')

    man_login_id_textbox.value = ""
    man_pw_textbox.value = ""

  login_button.on_click(on_login_button_clicked)
  man_back_button.on_click(man_back_button_clicked)

  man_login_labelVBox = widgets.VBox([man_login_id_label,man_pw_label])
  man_login_textboxVBox = widgets.VBox([man_login_id_textbox, man_pw_textbox])
  man_login_line1HBox = widgets.HBox([man_login_labelVBox, man_login_textboxVBox])
  man_login_line2HBox = widgets.HBox([login_button, man_back_button])

  display(widgets.VBox([man_login_line1HBox, man_login_line2HBox]))

#Category - lacking order function + sales volume
with tb_all.output_to('Category'):
  cat_grid = GridspecLayout(18, 2, height='600px', width='650px', grid_gap='10px')
  cat_title = widgets.Label('Viewing by Categories')
  cat_HistoryBtn = widgets.Button(description="Order History")
  cat_CartBtn = widgets.Button(description="Order Cart")
  cat_MainBtn = widgets.Button(description='Main Page')
  cat_SelectBtn = widgets.Button(description='Select')

  catls = []
  c.execute("SELECT F_Category FROM Menu WHERE f_availability = 'Available'")
  for row in c:
    catls.append(row[0])
  catls = set(catls)
  cat_dropdown = widgets.Dropdown(options = catls, value= None)
  def cat_MainBtnClick(b):
    showPage('Menu')
  def cat_HistoryBtnClick(b):
    global prevPage
    prevPage = 'Category'
    ordHis_display(getOrderHistory())
    showPage('Order History')
  def cat_CartBtnClick(b):
    global prevPage
    prevPage = 'Category'
    #insert ordercart display retrieve
    displayOrderCart(getOrderCart())
    showPage('Order Cart')
  cat_HistoryBtn.on_click(cat_HistoryBtnClick)
  cat_CartBtn.on_click(cat_CartBtnClick)
  cat_MainBtn.on_click(cat_MainBtnClick)
  catRow1_layout = widgets.Layout(display='center', flex_flow='col', justify_content='center', align_items='center', border='1px solid grey', height='35px')
  catRow1 = widgets.HBox([cat_CartBtn,cat_MainBtn, cat_HistoryBtn], layout=catRow1_layout)
  cat_grid[0,:] = catRow1

  catRow2_layout = widgets.Layout(display='center', flex_flow='col', align_items='center', border='1px solid grey')
  catRow2second = widgets.HBox([cat_dropdown,cat_SelectBtn])
  catRow2 = widgets.VBox([cat_title,catRow2second],layout=catRow2_layout)
  cat_grid[1:3,:] = catRow2

  def ind(pagenum,ls):
    mpp=4
    maximum = pagenum*mpp
    values = int()
    if len(ls)<maximum:
      maxrange = mpp - (maximum-len(ls))
      values = ls[-maxrange:]
    else:
      minr = (pagenum-1)*4
      maxr = (pagenum-1)*4+4
      values=ls[minr:maxr]
    return values

  quantity = [0,1,2,3,4,5,6,7,8,9,10]


  def displayItems(pageInfo):
    global item1_qty, item2_qty, item3_qty, item4_qty
    global item1_fid, item2_fid, item3_fid, item4_fid
    if len(pageInfo)>0:
      item1_layout = widgets.Layout(flex_flow='col', border='1px solid grey')
      item1_fname = widgets.Label(catitem1name)
      item1_fdesc = widgets.Label(catitem1desc)
      item1_fprice = widgets.Label(str(catitem1price))
      item1_fid = catitem1id
      item1_finventory = catitem1inv
      item1_fdesclabel = widgets.Label('Description: ')
      item1_fpricelabel = widgets.Label('Price: ')
      item1_fnamelabel = widgets.Label('Item Name: ')
      item1_qty = widgets.Dropdown(options=quantity, value=catitem1amt)
      def on_change(change):
        if change['type']=='change' and change['name']=='value':
          newQty=change['new']
          if not update_cart(item1_fid,newQty):
            show_message('Category','Order Cart Full')
            item1_qty.value = 0
      item1_qty.observe(on_change)
      item1_salesVol = widgets.Label(str(catitem1salesVol))
      item1_salesVollabel = widgets.Label('Sales Volume: ')
      it1_line4 = widgets.HBox([item1_salesVollabel, item1_salesVol])
      it1_line1 = widgets.HBox([item1_fnamelabel, item1_fname])
      it1_line2 = widgets.HBox([item1_fdesclabel, item1_fdesc])
      it1_line3 = widgets.HBox([item1_fpricelabel, item1_fprice])
      it1_line5 = widgets.HBox([item1_qty])
      item1 = widgets.VBox([it1_line1,it1_line2,it1_line3,it1_line4,it1_line5],layout=item1_layout)
      cat_grid[3:8,0:1] = item1

    if len(pageInfo)>1:
      item2_layout = widgets.Layout(flex_flow='col', border='1px solid grey')
      item2_fname = widgets.Label(catitem2name)
      item2_fdesc = widgets.Label(catitem2desc)
      item2_fprice = widgets.Label(str(catitem2price))
      item2_fid = catitem2id
      item2_finventory = catitem2inv
      item2_fdesclabel = widgets.Label('Description: ')
      item2_fpricelabel = widgets.Label('Price: ')
      item2_fnamelabel = widgets.Label('Item Name: ')
      item2_qty = widgets.Dropdown(options=quantity, value=catitem2amt)
      def on_change(change):
        if change['type']=='change' and change['name']=='value':
          newQty=change['new']
          if not update_cart(item2_fid,newQty):
            show_message('Category','Order Cart Full')
            item2_qty.value = 0
      item2_qty.observe(on_change)
      item2_salesVol = widgets.Label(str(catitem2salesVol))
      item2_salesVollabel = widgets.Label('Sales Volume: ')
      it2_line4 = widgets.HBox([item2_salesVollabel, item2_salesVol])
      it2_line1 = widgets.HBox([item2_fnamelabel, item2_fname])
      it2_line2 = widgets.HBox([item2_fdesclabel, item2_fdesc])
      it2_line3 = widgets.HBox([item2_fpricelabel, item2_fprice])
      it2_line5 = widgets.HBox([item2_qty])
      item2 = widgets.VBox([it2_line1,it2_line2,it2_line3,it2_line4,it2_line5],layout=item2_layout)
      cat_grid[3:8,1:] = item2

    if len(pageInfo)>2:
      item3_layout = widgets.Layout(flex_flow='col', border='1px solid grey')
      item3_fname = widgets.Label(catitem3name)
      item3_fdesc = widgets.Label(catitem3desc)
      item3_fprice = widgets.Label(str(catitem3price))
      item3_fid = catitem3id
      item3_finventory = catitem3inv
      item3_fdesclabel = widgets.Label('Description: ')
      item3_fpricelabel = widgets.Label('Price: ')
      item3_fnamelabel = widgets.Label('Item Name: ')
      item3_qty = widgets.Dropdown(options=quantity, value=catitem3amt)
      def on_change(change):
        if change['type']=='change' and change['name']=='value':
          newQty=change['new']
          if not update_cart(item3_fid,newQty):
            show_message('Category','Order Cart Full')
            item3_qty.value = 0
      item3_qty.observe(on_change)
      item3_salesVol = widgets.Label(str(catitem3salesVol))
      item3_salesVollabel = widgets.Label('Sales Volume: ')
      it3_line4 = widgets.HBox([item3_salesVollabel, item3_salesVol])
      it3_line1 = widgets.HBox([item3_fnamelabel, item3_fname])
      it3_line2 = widgets.HBox([item3_fdesclabel, item3_fdesc])
      it3_line3 = widgets.HBox([item3_fpricelabel, item3_fprice])
      it3_line5 = widgets.HBox([item3_qty])
      item3 = widgets.VBox([it3_line1,it3_line2,it3_line3,it3_line4,it3_line5],layout=item3_layout)
      cat_grid[8:13,:1] = item3

    if len(pageInfo)>3:
      item4_layout = widgets.Layout(flex_flow='col', border='1px solid grey')
      item4_fname = widgets.Label(catitem4name)
      item4_fdesc = widgets.Label(catitem4desc)
      item4_fprice = widgets.Label(str(catitem4price))
      item4_fid = catitem4id
      item4_finventory = catitem4inv
      item4_fdesclabel = widgets.Label('Description: ')
      item4_fpricelabel = widgets.Label('Price: ')
      item4_fnamelabel = widgets.Label('Item Name: ')
      item4_qty = widgets.Dropdown(options=quantity, value=catitem4amt)
      def on_change(change):
        if change['type']=='change' and change['name']=='value':
          newQty=change['new']
          if not update_cart(item4_fid,newQty):
            show_message('Category','Order Cart Full')
            item4_qty.value = 0
      item4_qty.observe(on_change)
      item4_salesVol = widgets.Label(str(catitem4salesVol))
      item4_salesVollabel = widgets.Label('Sales Volume: ')
      it4_line4 = widgets.HBox([item4_salesVollabel, item4_salesVol])
      it4_line1 = widgets.HBox([item4_fnamelabel, item4_fname])
      it4_line2 = widgets.HBox([item4_fdesclabel, item4_fdesc])
      it4_line3 = widgets.HBox([item4_fpricelabel, item4_fprice])
      it4_line5 = widgets.HBox([item4_qty])
      item4 = widgets.VBox([it4_line1,it4_line2,it4_line3,it4_line4,it4_line5],layout=item4_layout)
      cat_grid[8:13,1:] = item4


  def selectOnClick(b):
    global itemsDisplayed, pageItems, maxDisplayPage, categoryItems
    #clear existing layout
    emptyBoard()

    #getting max page
    currPage = 1
    maxDisplayPage = Label("")
    c.execute("SELECT count(*) FROM MENU WHERE f_category = '{}'".format(cat_dropdown.value))
    catProductCount = int()
    for a in c:
      catProductCount = int(a[0])
    maxDisplayPage.value = str(catProductCount//4)
    if catProductCount%4!=0:
      maxDisplayPage.value = str(int(catProductCount/4)+1)
    else:
      maxDisplayPage.value = str(int(catProductCount/4))
    pageTxt.value = 'Page {} out of {}'.format(currPage,maxDisplayPage.value)


    categoryItems=[k for k,v in allFoodItemsDt.items() if v[3] == cat_dropdown.value]
    #decide which page
    pageInfo = ind(currPage,categoryItems)
    pageItems = []
    for a in pageInfo:
      pageItems.append(allFoodItemsDt[a])

    catindex=1

    for a in range(len(pageItems)):
      naming = 'catitem'+str(catindex)
      globals()[naming+'id']=pageItems[a][0]
      globals()[naming+'name']=pageItems[a][1]
      globals()[naming+'desc']=pageItems[a][2]
      globals()[naming+'cat']=pageItems[a][3]
      globals()[naming+'price']=pageItems[a][4]
      globals()[naming+'avail']=pageItems[a][5]
      globals()[naming+'inv']=pageItems[a][6]
      if pageItems[a][0] in getOrderCart():
        globals()[naming+'amt']=getOrderCart()[pageItems[a][0]]
      else:
        globals()[naming+'amt']=0
      if catindex!=4:
        catindex+=1
      globals()[naming+'salesVol']=pageItems[a][8]
    #to display
    displayItems(pageInfo)

    showPage('Category')
  cat_SelectBtn.on_click(selectOnClick)

  #empty board
  def emptyBoard():
    item1_layout = widgets.Layout(flex_flow='col', border='1px solid grey')
    item1 = widgets.VBox([],layout=item1_layout)
    cat_grid[3:8,0:1] = item1
    item2_layout = widgets.Layout(flex_flow='col', border='1px solid grey')
    item2 = widgets.VBox([],layout=item2_layout)
    cat_grid[3:8,1:] = item2
    item3_layout = widgets.Layout(flex_flow='col', border='1px solid grey')
    item3 = widgets.VBox([],layout=item3_layout)
    cat_grid[8:13,:1] = item3
    item4_layout = widgets.Layout(flex_flow='col', border='1px solid grey')
    item4 = widgets.VBox([],layout=item4_layout)
    cat_grid[8:13,1:] = item4
  emptyBoard()

  currPage = 1
  maxDisplayPage = 0
  pageTxt = Label("")
  if maxDisplayPage == 0:
    pageTxt.value = 'Please select a category to continue'
  else:
    pageTxt.value = 'Page {} out of {}'.format(currPage,maxDisplayPage.value)

  cat_lastPage = widgets.Button(description='Last Page')
  cat_nextPage = widgets.Button(description='Next Page')

  def cat_lastPageClicked(b):
    global currPage, itemsDisplayed, pageItems
    if cat_dropdown.value != None:
      if int(maxDisplayPage.value) !=0 and int(maxDisplayPage.value) >= currPage > 1:
        #clear existing layout
        emptyBoard()
        #draw item
        currPage = currPage-1
        pageInfo = ind(currPage,categoryItems)
        pageItems = []
        for a in pageInfo:
          pageItems.append(allFoodItemsDt[a])
        catindex=1
        for a in range(len(pageItems)):
          naming = 'catitem'+str(catindex)
          catindex+=1
          globals()[naming+'id']=pageItems[a][0]
          globals()[naming+'name']=pageItems[a][1]
          globals()[naming+'desc']=pageItems[a][2]
          globals()[naming+'cat']=pageItems[a][3]
          globals()[naming+'price']=pageItems[a][4]
          globals()[naming+'avail']=pageItems[a][5]
          globals()[naming+'inv']=pageItems[a][6]
          if pageItems[a][0] in getOrderCart():
            globals()[naming+'amt']=getOrderCart()[pageItems[a][0]]
          else:
            globals()[naming+'amt']=0
          globals()[naming+'salesVol']=pageItems[a][8]
        displayItems(pageInfo)

      #change page num
      pageTxt.value = 'Page {} out of {}'.format(currPage,maxDisplayPage.value)

  def cat_nextPageClicked(b):
    global currPage, itemsDisplayed, pageItems
    if cat_dropdown.value != None:
      if int(maxDisplayPage.value) !=0 and int(maxDisplayPage.value) > currPage >= 1:
        #clear existing layout
        emptyBoard()
        #draw item
        #change current page
        currPage = currPage+1
        pageInfo = ind(currPage,categoryItems)
        pageItems = []
        for a in pageInfo:
          pageItems.append(allFoodItemsDt[a])
        catindex=1
        for a in range(len(pageItems)):
          naming = 'catitem'+str(catindex)
          catindex+=1
          globals()[naming+'id']=pageItems[a][0]
          globals()[naming+'name']=pageItems[a][1]
          globals()[naming+'desc']=pageItems[a][2]
          globals()[naming+'cat']=pageItems[a][3]
          globals()[naming+'price']=pageItems[a][4]
          globals()[naming+'avail']=pageItems[a][5]
          globals()[naming+'inv']=pageItems[a][6]
          if pageItems[a][0] in getOrderCart():
            globals()[naming+'amt']=getOrderCart()[pageItems[a][0]]
          else:
            globals()[naming+'amt']=0
          globals()[naming+'salesVol']=pageItems[a][8]
        displayItems(pageInfo)

      #change page num
      pageTxt.value = 'Page {} out of {}'.format(currPage,maxDisplayPage.value)

  cat_lastPage.on_click(cat_lastPageClicked)
  cat_nextPage.on_click(cat_nextPageClicked)

  page_layout = widgets.Layout(display='center', flex_flow='col', justify_content='center', align_items='center', border='1px solid grey')
  pageRow1 = widgets.HBox([cat_lastPage, pageTxt, cat_nextPage],layout=page_layout)
  page = widgets.VBox([pageRow1])
  cat_grid[13:,:] = page
  display(cat_grid)

#Order Cart
with tb_all.output_to('Order Cart'):
  ordercart = {}
  cart_grid = GridspecLayout(20, 4, height='650px', width='700px', grid_gap='5px')
  cart_text = widgets.Label('Order Cart')
  cartRow1_layout = widgets.Layout(display='center', flex_flow='col', justify_content='center', align_items='center', border='1px solid grey', height='35px')
  cartRow1 = widgets.HBox([cart_text], layout=catRow1_layout)
  cart_grid[0,:] = cartRow1

  #cart msg box
  if len(ordercart) == 0:
    cart_msgbox_text = widgets.Label('Cart is empty!')
  cart_msgbox_layout = widgets.Layout(display='center', flex_flow='col', justify_content='center', align_items='center', border='1px solid grey', height='35px')
  cart_msgbox_row = widgets.HBox([cart_msgbox_text],layout=cart_msgbox_layout)
  cart_grid[1:2,:]=cart_msgbox_row


  #cart item1-4
  def emptyCartDisplay():
    cart_item1_layout = cartItem1_layout = widgets.Layout( flex_flow='col', border='1px solid grey')
    cart_item1 = widgets.VBox([],layout=cart_item1_layout)
    cart_grid[2:6,:] =cart_item1
    cart_item2_layout = cartItem2_layout = widgets.Layout( flex_flow='col', border='1px solid grey')
    cart_item2 = widgets.VBox([],layout=cart_item2_layout)
    cart_grid[6:10,:] =cart_item2
    cart_item3_layout = cartItem3_layout = widgets.Layout( flex_flow='col', border='1px solid grey')
    cart_item3 = widgets.VBox([],layout=cart_item3_layout)
    cart_grid[10:14,:] =cart_item3
    cart_item4_layout = cartItem4_layout = widgets.Layout( flex_flow='col', border='1px solid grey')
    cart_item4 = widgets.VBox([],layout=cart_item4_layout)
    cart_grid[14:18,:] =cart_item4
  emptyCartDisplay()

  #bottom
  backToMenu_button = widgets.Button(description='Back To Menu')
  sendOrder_button = widgets.Button(description='Send Order')

  def backToMenuClicked(b):
    showPage('Menu')

  def coIDgenerator():
    existingCOID = c.execute("SELECT CO_ID FROM Cooking_Order")
    existingCOIDls = [a[0] for a in existingCOID]
    COIDgen = [str(random.randint(0,9)) for a in range(10)]
    COID = ''.join(COIDgen)
    while COID not in existingCOIDls:
      existingCOIDls.append(COID)
    else:
      COIDgen = [str(random.randint(0,9)) for a in range(10)]
      COID = ''.join(COIDgen)
      existingCOIDls.append(COID)
    return COID

  def displayOrderCart(ordCart):
    noOfItems = len(ordCart)
    if noOfItems == 0:
      cDO_msg = widgets.Label('Cart is empty!')
    else:
      cDO_msg = widgets.Label('You have {} item(s) in your order cart'.format(noOfItems))
    cDO_msg_layout = widgets.Layout(display='center', flex_flow='col', justify_content='center', align_items='center', border='1px solid grey', height='35px')
    cDO_msgBox = widgets.HBox([cDO_msg],layout=cDO_msg_layout)
    cart_grid[1:2,:]=cDO_msgBox
    emptyCartDisplay()
    ord = ordCart
    disItems = {k:allFoodItemsDt[k] for k,v in ord.items()}
    lsItems = list(disItems.keys())
    newQuantity = [1,2,3,4,5,6,7,8,9,10]
    if len(disItems)>0:
      item1 = lsItems[0]
      item1_NameLabel = widgets.Label('Name: ')
      item1_Name = widgets.Label(disItems[item1][1])
      item1_PriceLabel = widgets.Label('Price: $')
      item1_Price = widgets.Label(str(disItems[item1][4]))
      item1_QtyLabel = widgets.Label('Quantity:')
      item1_Qty = widgets.Dropdown(options = newQuantity, value = ord[item1])
      def on_change(change):
        if change['type']=='change' and change['name']=='value':
          newQty=change['new']
          if not update_cart(item1,newQty):
            show_message('Category','Order Cart Full')
            item1_Qty.value = 0
      item1_Qty.observe(on_change)
      item1_RemoveItem = widgets.Button(description='Remove Item')
      def removeItem(b):
        update_cart(item1,0)
        displayOrderCart(getOrderCart())
      item1_RemoveItem.on_click(removeItem)
      item1_Line1 = widgets.HBox([item1_NameLabel,item1_Name])
      item1_Line2 = widgets.HBox([item1_PriceLabel, item1_Price])
      item1_Line3 = widgets.HBox([item1_QtyLabel, item1_Qty])
      item1_Line4 = widgets.HBox([item1_RemoveItem])
      item1_food_cart_layout = widgets.Layout( flex_flow='col', border='1px solid grey')
      item1_VBox = widgets.VBox([item1_Line1,item1_Line2,item1_Line3,item1_Line4], layout = item1_food_cart_layout)
      cart_grid[2:6,:] = item1_VBox
    if len(disItems)>1:
      item2 = lsItems[1]
      item2_NameLabel = widgets.Label('Name: ')
      item2_Name = widgets.Label(disItems[item2][1])
      item2_PriceLabel = widgets.Label('Price: $')
      item2_Price = widgets.Label(str(disItems[item2][4]))
      item2_QtyLabel = widgets.Label('Quantity:')
      item2_Qty = widgets.Dropdown(options = newQuantity, value = ord[item2])
      def on_change(change):
        if change['type']=='change' and change['name']=='value':
          newQty=change['new']
          if not update_cart(item2,newQty):
            show_message('Category','Order Cart Full')
            item2_Qty.value = 0
      item2_Qty.observe(on_change)
      item2_RemoveItem = widgets.Button(description='Remove Item')
      def removeItem(b):
        update_cart(item2,0)
        displayOrderCart(getOrderCart())
      item2_RemoveItem.on_click(removeItem)
      item2_Line1 = widgets.HBox([item2_NameLabel,item2_Name])
      item2_Line2 = widgets.HBox([item2_PriceLabel, item2_Price])
      item2_Line3 = widgets.HBox([item2_QtyLabel, item2_Qty])
      item2_Line4 = widgets.HBox([item2_RemoveItem])
      item2_food_cart_layout = widgets.Layout( flex_flow='col', border='1px solid grey')
      item2_VBox = widgets.VBox([item2_Line1,item2_Line2,item2_Line3,item2_Line4], layout = item2_food_cart_layout)
      item2_RemoveItemBox = widgets.HBox([item2_RemoveItem])
      item2_RemoveItemLayout = widgets.Layout( flex_flow='col', border='1px solid grey')
      item2_RemItBox = widgets.VBox([item2_RemoveItemBox],layout=item2_RemoveItemLayout)
      cart_grid[6:10,:] = item2_VBox
    if len(disItems)>2:
      item3 = lsItems[2]
      item3_NameLabel = widgets.Label('Name: ')
      item3_Name = widgets.Label(disItems[item3][1])
      item3_PriceLabel = widgets.Label('Price: $')
      item3_Price = widgets.Label(str(disItems[item3][4]))
      item3_QtyLabel = widgets.Label('Quantity:')
      item3_Qty = widgets.Dropdown(options = newQuantity, value = ord[item3])
      def on_change(change):
        if change['type']=='change' and change['name']=='value':
          newQty=change['new']
          if not update_cart(item3,newQty):
            show_message('Category','Order Cart Full')
            item3_Qty.value = 0
      item3_Qty.observe(on_change)
      item3_RemoveItem = widgets.Button(description='Remove Item')
      def removeItem(b):
        update_cart(item3,0)
        displayOrderCart(getOrderCart())
      item3_RemoveItem.on_click(removeItem)
      item3_Line1 = widgets.HBox([item3_NameLabel,item3_Name])
      item3_Line2 = widgets.HBox([item3_PriceLabel, item3_Price])
      item3_Line3 = widgets.HBox([item3_QtyLabel, item3_Qty])
      item3_Line4 = widgets.HBox([item3_RemoveItem])
      item3_food_cart_layout = widgets.Layout( flex_flow='col', border='1px solid grey')
      item3_VBox = widgets.VBox([item3_Line1,item3_Line2,item3_Line3,item3_Line4], layout = item3_food_cart_layout)
      item3_RemoveItemBox = widgets.HBox([item3_RemoveItem])
      item3_RemoveItemLayout = widgets.Layout( flex_flow='col', border='1px solid grey')
      item3_RemItBox = widgets.VBox([item3_RemoveItemBox],layout=item3_RemoveItemLayout)
      cart_grid[10:14,:] = item3_VBox
    if len(disItems)>3:
      item4 = lsItems[3]
      item4_NameLabel = widgets.Label('Name: ')
      item4_Name = widgets.Label(disItems[item4][1])
      item4_PriceLabel = widgets.Label('Price: $')
      item4_Price = widgets.Label(str(disItems[item4][4]))
      item4_QtyLabel = widgets.Label('Quantity:')
      item4_Qty = widgets.Dropdown(options = newQuantity, value = ord[item4])
      def on_change(change):
        if change['type']=='change' and change['name']=='value':
          newQty=change['new']
          if not update_cart(item4,newQty):
            show_message('Category','Order Cart Full')
            item4_Qty.value = 0
      item4_Qty.observe(on_change)
      item4_RemoveItem = widgets.Button(description='Remove Item')
      def removeItem(b):
        update_cart(item4,0)
        displayOrderCart(getOrderCart())
      item4_RemoveItem.on_click(removeItem)
      item4_Line1 = widgets.HBox([item4_NameLabel,item4_Name])
      item4_Line2 = widgets.HBox([item4_PriceLabel, item4_Price])
      item4_Line3 = widgets.HBox([item4_QtyLabel, item4_Qty])
      item4_Line4 = widgets.HBox([item4_RemoveItem])
      item4_food_cart_layout = widgets.Layout( flex_flow='col', border='1px solid grey')
      item4_VBox = widgets.VBox([item4_Line1,item4_Line2,item4_Line3,item4_Line4], layout = item4_food_cart_layout)
      item4_RemoveItemBox = widgets.HBox([item4_RemoveItem])
      item4_RemoveItemLayout = widgets.Layout( flex_flow='col', border='1px solid grey')
      item4_RemItBox = widgets.VBox([item4_RemoveItemBox],layout=item4_RemoveItemLayout)
      cart_grid[14:18,:] = item4_VBox
  displayOrderCart(getOrderCart())

  #sending order
  def coIDgenerator():
    existingCOID = c.execute("SELECT CO_ID FROM Cooking_Order")
    existingCOIDls = [a[0] for a in existingCOID]
    COIDgen = [str(random.randint(0,9)) for a in range(10)]
    COID = ''.join(COIDgen)
    while COID not in existingCOIDls:
      existingCOIDls.append(COID)
    else:
      COIDgen = [str(random.randint(0,9)) for a in range(10)]
      COID = ''.join(COIDgen)
      existingCOIDls.append(COID)
    return COID

  def getDbData(ordCart):
    store = {}
    for a in list(ordCart):
      c.execute("SELECT f_id, f_inventory, f_availability FROM MENU WHERE f_id = '{}'".format(a))
      for b in c:
        store[b[0]] = [b[1],b[2]]
    return store

  def sendOrderClicked(b):
    global prevPage
    ordCart = getOrderCart()
    prevPage = 'Order Cart'
    databaseInfo = getDbData(ordCart)
    #sunny day
    canOrder = []
    #rainy day
    cannotInvOrder = {}
    cannotAvOrder = []
    #filter items from the order cart into 3 categories
    for k,v in ordCart.items():
      if databaseInfo[k][1] == 'Unavailable':
        cannotAvOrder.append(k)
      elif v > databaseInfo[k][0]:
        cannotInvOrder[k]=databaseInfo[k][0]
      else:
        canOrder.append(k)

    if len(cannotAvOrder)>0:
      def unavStr(unav):
        strformat='Sorry,'
        for a in unav:
          foodname=allFoodItemsDt[a]
          strformat+=' item {} is unavailable, please remove it from your shopping cart'.format(foodname)
        return strformat
      messageBox_label.value=unavStr(cannotAvOrder)
      showPage('Message Box')
    elif len(cannotInvOrder)>0:
      def lackInvStr(unavailable):
        strformat='Sorry,'
        for k,v in unavailable.items():
          foodname = allFoodItemsDt[k][1]
          remainValue = v
          if v>0:
            strformat += ' item {} is not available, we only have {} quantity left'.format(foodname,remainValue)
          else:
            strformat += ' item {} is not available, it is out of stock'.format(foodname)
        return strformat
      messageBox_label.value=lackInvStr(cannotInvOrder)
      showPage('Message Box')
    elif len(ordCart)==0:
      messageBox_label.value='Empty cart, please select food items before sending order'
      showPage('Message Box')
    else:
      #1 update the menu f_inventory
      updateDbDt={}
      for k,v in ordCart.items():
        updateDbDt[k]= databaseInfo[k][0]-v
      for k,v in updateDbDt.items():
        c.execute("UPDATE Menu SET f_inventory = {} WHERE f_id = '{}'".format(v,k))
        c.execute('commit')
      #2 log cooking order
      for k,v in ordCart.items():
        coID = coIDgenerator()
        c.execute("""INSERT INTO cooking_order VALUES ('{}',{},'{}','',SYSDATE,'{}','{}')""".format(coID, v, 'Submitted',transID,k))
        c.execute('commit')
      #3 refresh the menu items
      global allfName, allfDesc,allfCat, allfPrice, allfAvail, allfID, allfInv, categoryItems, allFoodItemsDt
      allfName, allfDesc,allfCat, allfPrice, allfAvail, allfID, allfInv = [],[],[],[],[],[],[]
      c.execute("SELECT f_name,f_description,f_category,f_price,f_availability,f_id,f_inventory FROM MENU WHERE f_availability = 'Available'")
      for a in c:
        allfName.append(a[0])
        allfDesc.append(a[1])
        allfCat.append(a[2])
        allfPrice.append(a[3])
        allfAvail.append(a[4])
        allfID.append(a[5])
        allfInv.append(a[6])
      allFoodItemsDt={}
      for a in range(len(allfID)):
        allFoodItemsDt[allfID[a]] = [allfID[a],allfName[a],allfDesc[a],allfCat[a],allfPrice[a],allfAvail[a],allfInv[a],0]
      c.execute("SELECT sum(co_qty), menu_f_id FROM cooking_order GROUP BY menu_f_id")
      salesVolume={a[1]:a[0] for a in c}
      for k,v in allFoodItemsDt.items():
        if k in salesVolume:
          v.append(salesVolume[k])
        else:
          v.append(0)
      #4 reset cart
      resetCart()
      #5 display 'your order has been placed'
      prevPage = 'Menu'
      messageBox_label.value='Your order has been placed'
      showPage('Message Box')

  sendOrder_button.on_click(sendOrderClicked)
  backToMenu_button.on_click(backToMenuClicked)

  bottomButtons_layout = widgets.Layout(display='center', flex_flow='col', justify_content='center', align_items='center', border='1px solid grey', height='35px')
  bottomRow = widgets.HBox([backToMenu_button,sendOrder_button],layout = bottomButtons_layout)
  cart_grid[18:,:]=bottomRow

  display(cart_grid)

with tb_all.output_to('Order History'):
  orderHistory = {}
  ordHis_grid = GridspecLayout(26, 4, height='650px', width='700px', grid_gap='5px')
  ordHis_text = widgets.Label('Order History')
  ordHisRow1_layout = widgets.Layout(display='center', flex_flow='col', justify_content='center', align_items='center', border='1px solid grey', height='35px')
  ordHisRow1 = widgets.HBox([ordHis_text], layout=ordHisRow1_layout)
  ordHis_grid[0,:] = ordHisRow1

  ordHis_refreshButton = widgets.Button(description='Refresh Page')
  ordHisRow2_layout = widgets.Layout(display='center', flex_flow='col', justify_content='center', align_items='center', border='1px solid grey', height='35px')
  ordHisRow2 = widgets.HBox([ordHis_refreshButton], layout=ordHisRow2_layout)
  ordHis_grid[1,:] = ordHisRow2

  orderHis_currPage=1
  ohPageTxt = Label("")
  #problem with page number
  if ohMaxDisplayPage == 0:
    ohPageTxt.value = 'No orders yet'
  else:
    ohPageTxt.value = 'Page {} out of {}'.format(str(orderHis_currPage),str(ohMaxDisplayPage))

  ohPageTxt_layout = widgets.Layout(display='center', flex_flow='col', justify_content='center', align_items='center', border='1px solid grey')
  ohPageTxtRow = widgets.HBox([ohPageTxt],layout=ohPageTxt_layout)
  ohPageTxtitem = widgets.VBox([ohPageTxtRow])
  ordHis_grid[22:24,:] = ohPageTxtitem

  def refreshOnClick(b):
    ordHis_display(getOrderHistory())
  ordHis_refreshButton.on_click(refreshOnClick)

  def emptyOrdHisDisplay():
    ordHis_item1_layout = widgets.Layout( flex_flow='col', border='1px solid grey')
    ordHis_item1 = widgets.VBox([],layout=ordHis_item1_layout)
    ordHis_grid[2:7,:] =ordHis_item1
    ordHis_item2_layout = widgets.Layout( flex_flow='col', border='1px solid grey')
    ordHis_item2 = widgets.VBox([],layout=ordHis_item2_layout)
    ordHis_grid[7:12,:] =ordHis_item2
    ordHis_item3_layout = widgets.Layout( flex_flow='col', border='1px solid grey')
    ordHis_item3 = widgets.VBox([],layout=ordHis_item3_layout)
    ordHis_grid[12:17,:] =ordHis_item3
    ordHis_item4_layout = widgets.Layout( flex_flow='col', border='1px solid grey')
    ordHis_item4 = widgets.VBox([],layout=ordHis_item4_layout)
    ordHis_grid[17:22,:] =ordHis_item4

  emptyOrdHisDisplay()


  def ohLastPageClick(b):
    global orderHis_currPage
    if len(orderHisSorted)%4!=0:
      maxPage = (len(orderHisSorted)//4)+1
    else:
      maxPage = len(orderHisSorted)//4
    if orderHis_currPage!=1:
      orderHis_currPage -=1
      ordHis_display(getOrderHistory())
      ohPageTxt.value = 'Page {} out of {}'.format(str(orderHis_currPage),str(ohMaxDisplayPage))

  def ohNextPageClick(b):
    global orderHis_currPage
    if len(orderHisSorted)%4!=0:
      maxPage = (len(orderHisSorted)//4)+1
    else:
      maxPage = len(orderHisSorted)//4
    if orderHis_currPage < maxPage:
      orderHis_currPage+=1
      if len(orderHisSorted)%4!=0:
        ohMaxDisplayPage = (len(orderHisSorted)//4)+1
      else:
        ohMaxDisplayPage = len(orderHisSorted)//4
      ordHis_display(getOrderHistory())
      ohPageTxt.value = 'Page {} out of {}'.format(str(orderHis_currPage),str(ohMaxDisplayPage))

  def mainPageClick(b):
    showPage('Menu')
  lastPageBtn = widgets.Button(description='Last Page')
  nextPageBtn = widgets.Button(description='Next Page')
  mainPageBtn = widgets.Button(description='Main Page')
  lastPageBtn.on_click(ohLastPageClick)
  nextPageBtn.on_click(ohNextPageClick)
  mainPageBtn.on_click(mainPageClick)

  ohbottom_layout = widgets.Layout(display='center', flex_flow='col', justify_content='center', align_items='center', border='1px solid grey')
  ohbottomRow = widgets.HBox([lastPageBtn,mainPageBtn,nextPageBtn],layout=ohbottom_layout)
  ohbottom = widgets.VBox([ohbottomRow])
  ordHis_grid[24:,:] = ohbottom

  def ordHis_display(itemls):
    global orderHis,orderHis_currPage,ohMaxDisplayPage
    if ohMaxDisplayPage == 0:
      ohPageTxt.value = 'No orders yet'
    else:
      ohPageTxt.value = f'Page {str(orderHis_currPage)} out of {str(ohMaxDisplayPage)}'
    emptyOrdHisDisplay()
    if len(itemls)>0:
      ordHis_item1 = itemls[0]
      ordHis_item1timelabel = widgets.Label('Order Time: ')
      ordHis_item1fnamelabel = widgets.Label('Food Name: ')
      ordHis_item1coqtylabel = widgets.Label('Order Quantity: ')
      ordHis_item1fpricelabel = widgets.Label('Item Price: ')
      ordHis_item1statuslabel = widgets.Label('Item Status: ')

      ordHis_item1time = widgets.Label(str(orderHis[ordHis_item1]['co_submission_time'].strftime('%H:%M %Y-%m-%d')))
      ordHis_item1fname = widgets.Label(orderHis[ordHis_item1]['f_name'])
      ordHis_item1coqty = widgets.Label(str(orderHis[ordHis_item1]['co_qty']))
      ordHis_item1fprice = widgets.Label(str(orderHis[ordHis_item1]['f_price']))
      ordHis_item1status = widgets.Label(orderHis[ordHis_item1]['co_status'])

      ordHis_item1_Line1 = widgets.HBox([ordHis_item1timelabel,ordHis_item1time])
      ordHis_item1_Line2 = widgets.HBox([ordHis_item1fnamelabel, ordHis_item1fname ])
      ordHis_item1_Line3 = widgets.HBox([ordHis_item1coqtylabel, ordHis_item1coqty])
      ordHis_item1_Line4 = widgets.HBox([ordHis_item1fpricelabel,ordHis_item1fprice])
      ordHis_item1_Line5 = widgets.HBox([ordHis_item1statuslabel,ordHis_item1status])
      ordHis_item1_food_cart_layout = widgets.Layout( flex_flow='col', border='1px solid grey')
      ordHis_item1_VBox = widgets.VBox([ordHis_item1_Line1,ordHis_item1_Line2,ordHis_item1_Line3,ordHis_item1_Line4,ordHis_item1_Line5], layout = ordHis_item1_food_cart_layout)
      ordHis_grid[2:7,:] = ordHis_item1_VBox
    if len(itemls)>1:
      ordHis_item2 = itemls[1]
      ordHis_item2timelabel = widgets.Label('Order Time: ')
      ordHis_item2fnamelabel = widgets.Label('Food Name: ')
      ordHis_item2coqtylabel = widgets.Label('Order Quantity: ')
      ordHis_item2fpricelabel = widgets.Label('Item Price: ')
      ordHis_item2statuslabel = widgets.Label('Item Status: ')

      ordHis_item2time = widgets.Label(str(orderHis[ordHis_item2]['co_submission_time'].strftime('%H:%M %Y-%m-%d')))
      ordHis_item2fname = widgets.Label(orderHis[ordHis_item2]['f_name'])
      ordHis_item2coqty = widgets.Label(str(orderHis[ordHis_item2]['co_qty']))
      ordHis_item2fprice = widgets.Label(str(orderHis[ordHis_item2]['f_price']))
      ordHis_item2status = widgets.Label(orderHis[ordHis_item2]['co_status'])

      ordHis_item2_Line1 = widgets.HBox([ordHis_item2timelabel,ordHis_item2time])
      ordHis_item2_Line2 = widgets.HBox([ordHis_item2fnamelabel, ordHis_item2fname ])
      ordHis_item2_Line3 = widgets.HBox([ordHis_item2coqtylabel, ordHis_item2coqty])
      ordHis_item2_Line4 = widgets.HBox([ordHis_item2fpricelabel,ordHis_item2fprice])
      ordHis_item2_Line5 = widgets.HBox([ordHis_item2statuslabel,ordHis_item2status])
      ordHis_item2_food_cart_layout = widgets.Layout( flex_flow='col', border='1px solid grey')
      ordHis_item2_VBox = widgets.VBox([ordHis_item2_Line1,ordHis_item2_Line2,ordHis_item2_Line3,ordHis_item2_Line4,ordHis_item2_Line5], layout = ordHis_item2_food_cart_layout)
      ordHis_grid[7:12,:] = ordHis_item2_VBox
    if len(itemls)>2:
      ordHis_item3 = itemls[2]
      ordHis_item3timelabel = widgets.Label('Order Time: ')
      ordHis_item3fnamelabel = widgets.Label('Food Name: ')
      ordHis_item3coqtylabel = widgets.Label('Order Quantity: ')
      ordHis_item3fpricelabel = widgets.Label('Item Price: ')
      ordHis_item3statuslabel = widgets.Label('Item Status: ')

      ordHis_item3time = widgets.Label(str(orderHis[ordHis_item3]['co_submission_time'].strftime('%H:%M %Y-%m-%d')))
      ordHis_item3fname = widgets.Label(orderHis[ordHis_item3]['f_name'])
      ordHis_item3coqty = widgets.Label(str(orderHis[ordHis_item3]['co_qty']))
      ordHis_item3fprice = widgets.Label(str(orderHis[ordHis_item3]['f_price']))
      ordHis_item3status = widgets.Label(orderHis[ordHis_item3]['co_status'])

      ordHis_item3_Line1 = widgets.HBox([ordHis_item3timelabel,ordHis_item3time])
      ordHis_item3_Line2 = widgets.HBox([ordHis_item3fnamelabel, ordHis_item3fname ])
      ordHis_item3_Line3 = widgets.HBox([ordHis_item3coqtylabel, ordHis_item3coqty])
      ordHis_item3_Line4 = widgets.HBox([ordHis_item3fpricelabel,ordHis_item3fprice])
      ordHis_item3_Line5 = widgets.HBox([ordHis_item3statuslabel,ordHis_item3status])
      ordHis_item3_food_cart_layout = widgets.Layout( flex_flow='col', border='1px solid grey')
      ordHis_item3_VBox = widgets.VBox([ordHis_item3_Line1,ordHis_item3_Line2,ordHis_item3_Line3,ordHis_item3_Line4,ordHis_item3_Line5], layout = ordHis_item3_food_cart_layout)
      ordHis_grid[12:17,:] = ordHis_item3_VBox
    if len(itemls)>3:
      ordHis_item4 = itemls[3]
      ordHis_item4timelabel = widgets.Label('Order Time: ')
      ordHis_item4fnamelabel = widgets.Label('Food Name: ')
      ordHis_item4coqtylabel = widgets.Label('Order Quantity: ')
      ordHis_item4fpricelabel = widgets.Label('Item Price: ')
      ordHis_item4statuslabel = widgets.Label('Item Status: ')

      ordHis_item4time = widgets.Label(str(orderHis[ordHis_item4]['co_submission_time'].strftime('%H:%M %Y-%m-%d')))
      ordHis_item4fname = widgets.Label(orderHis[ordHis_item4]['f_name'])
      ordHis_item4coqty = widgets.Label(str(orderHis[ordHis_item4]['co_qty']))
      ordHis_item4fprice = widgets.Label(str(orderHis[ordHis_item4]['f_price']))
      ordHis_item4status = widgets.Label(orderHis[ordHis_item4]['co_status'])

      ordHis_item4_Line1 = widgets.HBox([ordHis_item4timelabel,ordHis_item4time])
      ordHis_item4_Line2 = widgets.HBox([ordHis_item4fnamelabel, ordHis_item4fname ])
      ordHis_item4_Line3 = widgets.HBox([ordHis_item4coqtylabel, ordHis_item4coqty])
      ordHis_item4_Line4 = widgets.HBox([ordHis_item4fpricelabel,ordHis_item4fprice])
      ordHis_item4_Line5 = widgets.HBox([ordHis_item4statuslabel,ordHis_item4status])
      ordHis_item4_food_cart_layout = widgets.Layout( flex_flow='col', border='1px solid grey')
      ordHis_item4_VBox = widgets.VBox([ordHis_item4_Line1,ordHis_item4_Line2,ordHis_item4_Line3,ordHis_item4_Line4,ordHis_item4_Line5], layout = ordHis_item4_food_cart_layout)
      ordHis_grid[17:22,:] = ordHis_item4_VBox


  display(ordHis_grid)

with tb_all.output_to('Popular'):
  pop_grid = GridspecLayout(37, 2, height='900px', width='700px', grid_gap='5px')
  pop_HistoryBtn = widgets.Button(description="Order History")
  pop_CartBtn = widgets.Button(description="Order Cart")
  pop_MainBtn = widgets.Button(description='Main Page')
  def pop_MainBtnClick(b):
    showPage('Menu')
  def pop_HistoryBtnClick(b):
    global prevPage
    prevPage = 'Popular'
    showPage('Order History')
  def pop_CartBtnClick(b):
    global prevPage
    prevPage = 'Popular'
    #insert ordercart display retrieve
    displayOrderCart(getOrderCart())
    showPage('Order Cart')
  def get_top_5_popular_items():
    c.execute("SELECT sum(co_qty), menu_f_id FROM cooking_order GROUP BY menu_f_id")
    salesVolume = {a[1]:a[0] for a in c if a[1] in allFoodItemsDt}
    maxFive = {k: salesVolume[k] for k in list(salesVolume)[:5]}
    sorted_salesVolume = sorted(maxFive.items(), key=lambda x:x[1],reverse=True)
    return sorted_salesVolume
  pop_HistoryBtn.on_click(pop_HistoryBtnClick)
  pop_CartBtn.on_click(pop_CartBtnClick)
  pop_MainBtn.on_click(pop_MainBtnClick)
  popRow1_layout = widgets.Layout(display='center', flex_flow='col', justify_content='center', align_items='center', border='1px solid grey', height='35px')
  popRow1 = widgets.HBox([pop_CartBtn,pop_MainBtn, pop_HistoryBtn], layout=popRow1_layout)
  pop_grid[0,:] = popRow1

  popRow2_Label = widgets.Label('Top 5 Popular Items')
  popRow2_layout = widgets.Layout(display='center', flex_flow='col', justify_content='center', align_items='center', border='1px solid grey', height='35px')
  popRow2 = widgets.HBox([popRow2_Label], layout=popRow2_layout)
  pop_grid[1,:] = popRow2

  def emptyPopGrid():
    pop_item1_layout = widgets.Layout(flex_flow='col', border='1px solid grey')
    pop_item1 = widgets.VBox([],layout=pop_item1_layout)
    pop_grid[2:9,:] = pop_item1
    pop_item2_layout = widgets.Layout(flex_flow='col', border='1px solid grey')
    pop_item2 = widgets.VBox([],layout=pop_item2_layout)
    pop_grid[9:16,:] = pop_item2
    pop_item3_layout = widgets.Layout(flex_flow='col', border='1px solid grey')
    pop_item3 = widgets.VBox([],layout=pop_item3_layout)
    pop_grid[16:23,:] = pop_item3
    pop_item4_layout = widgets.Layout(flex_flow='col', border='1px solid grey')
    pop_item4 = widgets.VBox([],layout=pop_item4_layout)
    pop_grid[23:30,:] = pop_item4
    pop_item5_layout = widgets.Layout(flex_flow='col', border='1px solid grey')
    pop_item5 = widgets.VBox([],layout=pop_item5_layout)
    pop_grid[30:37,:] = pop_item5
  emptyPopGrid()

  def displayPopItems(tup):
    popRow2_Label.value = f'Top {len(tup)} Popular Items'
    quantity = [0,1,2,3,4,5,6,7,8,9,10]
    if len(tup)>0:
      popItem1 = tup[0][0]
      popItem1_label = widgets.Label('Number 1')
      popItem1_fName_label = widgets.Label('Food Name: ')
      popItem1_fDesc_label = widgets.Label('Description: ')
      popItem1_fPrice_label = widgets.Label('Food Price: ')
      popItem1_fSV_label = widgets.Label('Sales Volume: ')

      popItem1_fName = widgets.Label(allFoodItemsDt[popItem1][1])
      popItem1_fDesc = widgets.Label(allFoodItemsDt[popItem1][2])
      popItem1_fPrice = widgets.Label(str(allFoodItemsDt[popItem1][4]))
      popItem1_fSV = widgets.Label(str(allFoodItemsDt[popItem1][8]))
      #update the dropdown value
      popItem1_ordQty = 0
      if popItem1 in getOrderCart():
        popItem1_ordQty=getOrderCart()[popItem1]

      popItem1_qty = widgets.Dropdown(options=quantity,value = popItem1_ordQty)
      def on_change(change):
        if change['type']=='change' and change['name']=='value':
          newQty=change['new']
          if not update_cart(popItem1,newQty):
            show_message('Category','Order Cart Full')
            popItem1.value = 0
      popItem1_qty.observe(on_change)

      popItem1_Line1 = widgets.HBox([popItem1_label])
      popItem1_Line2 = widgets.HBox([popItem1_fName_label, popItem1_fName])
      popItem1_Line3 = widgets.HBox([popItem1_fDesc_label, popItem1_fDesc])
      popItem1_Line4 = widgets.HBox([popItem1_fPrice_label,popItem1_fPrice])
      popItem1_Line5 = widgets.HBox([popItem1_fSV_label,popItem1_fSV])
      popItem1_Line6 = widgets.HBox([popItem1_qty])
      popItem1_layout = widgets.Layout(flex_flow='col', border='1px solid grey')
      popItem1_VBox = widgets.VBox([popItem1_Line1,popItem1_Line2,popItem1_Line3,popItem1_Line4,popItem1_Line5,popItem1_Line6], layout = popItem1_layout)
      pop_grid[2:9,:] = popItem1_VBox

    if len(tup)>1:
      popItem2 = tup[1][0]
      popItem2_label = widgets.Label('Number 2')
      popItem2_fName_label = widgets.Label('Food Name: ')
      popItem2_fDesc_label = widgets.Label('Description: ')
      popItem2_fPrice_label = widgets.Label('Food Price: ')
      popItem2_fSV_label = widgets.Label('Sales Volume: ')

      popItem2_fName = widgets.Label(allFoodItemsDt[popItem2][1])
      popItem2_fDesc = widgets.Label(allFoodItemsDt[popItem2][2])
      popItem2_fPrice = widgets.Label(str(allFoodItemsDt[popItem2][4]))
      popItem2_fSV = widgets.Label(str(allFoodItemsDt[popItem2][8]))
      #update the dropdown value
      popItem2_ordQty = 0
      if popItem2 in getOrderCart():
        popItem2_ordQty=getOrderCart()[popItem2]

      popItem2_qty = widgets.Dropdown(options=quantity,value = popItem2_ordQty)
      def on_change(change):
        if change['type']=='change' and change['name']=='value':
          newQty=change['new']
          if not update_cart(popItem2,newQty):
            show_message('Category','Order Cart Full')
            popItem2.value = 0
      popItem2_qty.observe(on_change)

      popItem2_Line1 = widgets.HBox([popItem2_label])
      popItem2_Line2 = widgets.HBox([popItem2_fName_label, popItem2_fName])
      popItem2_Line3 = widgets.HBox([popItem2_fDesc_label, popItem2_fDesc])
      popItem2_Line4 = widgets.HBox([popItem2_fPrice_label,popItem2_fPrice])
      popItem2_Line5 = widgets.HBox([popItem2_fSV_label,popItem2_fSV])
      popItem2_Line6 = widgets.HBox([popItem2_qty])
      popItem2_layout = widgets.Layout(flex_flow='col', border='1px solid grey')
      popItem2_VBox = widgets.VBox([popItem2_Line1,popItem2_Line2,popItem2_Line3,popItem2_Line4,popItem2_Line5,popItem2_Line6], layout = popItem2_layout)
      pop_grid[9:16,:] = popItem2_VBox

    if len(tup)>2:
      popItem3 = tup[2][0]
      popItem3_label = widgets.Label('Number 3')
      popItem3_fName_label = widgets.Label('Food Name: ')
      popItem3_fDesc_label = widgets.Label('Description: ')
      popItem3_fPrice_label = widgets.Label('Food Price: ')
      popItem3_fSV_label = widgets.Label('Sales Volume: ')

      popItem3_fName = widgets.Label(allFoodItemsDt[popItem3][1])
      popItem3_fDesc = widgets.Label(allFoodItemsDt[popItem3][2])
      popItem3_fPrice = widgets.Label(str(allFoodItemsDt[popItem3][4]))
      popItem3_fSV = widgets.Label(str(allFoodItemsDt[popItem3][8]))
      #update the dropdown value
      popItem3_ordQty = 0
      if popItem3 in getOrderCart():
        popItem3_ordQty=getOrderCart()[popItem3]

      popItem3_qty = widgets.Dropdown(options=quantity,value = popItem3_ordQty)
      def on_change(change):
        if change['type']=='change' and change['name']=='value':
          newQty=change['new']
          if not update_cart(popItem3,newQty):
            show_message('Category','Order Cart Full')
            popItem3.value = 0
      popItem3_qty.observe(on_change)

      popItem3_Line1 = widgets.HBox([popItem3_label])
      popItem3_Line2 = widgets.HBox([popItem3_fName_label, popItem3_fName])
      popItem3_Line3 = widgets.HBox([popItem3_fDesc_label, popItem3_fDesc])
      popItem3_Line4 = widgets.HBox([popItem3_fPrice_label,popItem3_fPrice])
      popItem3_Line5 = widgets.HBox([popItem3_fSV_label,popItem3_fSV])
      popItem3_Line6 = widgets.HBox([popItem3_qty])
      popItem3_layout = widgets.Layout(flex_flow='col', border='1px solid grey')
      popItem3_VBox = widgets.VBox([popItem3_Line1,popItem3_Line2,popItem3_Line3,popItem3_Line4,popItem3_Line5,popItem3_Line6], layout = popItem3_layout)
      pop_grid[16:23,:] = popItem3_VBox

    if len(tup)>3:
      popItem4 = tup[3][0]
      popItem4_label = widgets.Label('Number 4')
      popItem4_fName_label = widgets.Label('Food Name: ')
      popItem4_fDesc_label = widgets.Label('Description: ')
      popItem4_fPrice_label = widgets.Label('Food Price: ')
      popItem4_fSV_label = widgets.Label('Sales Volume: ')

      popItem4_fName = widgets.Label(allFoodItemsDt[popItem4][1])
      popItem4_fDesc = widgets.Label(allFoodItemsDt[popItem4][2])
      popItem4_fPrice = widgets.Label(str(allFoodItemsDt[popItem4][4]))
      popItem4_fSV = widgets.Label(str(allFoodItemsDt[popItem4][8]))
      #update the dropdown value
      popItem4_ordQty = 0
      if popItem4 in getOrderCart():
        popItem4_ordQty=getOrderCart()[popItem4]

      popItem4_qty = widgets.Dropdown(options=quantity,value = popItem4_ordQty)
      def on_change(change):
        if change['type']=='change' and change['name']=='value':
          newQty=change['new']
          if not update_cart(popItem4,newQty):
            show_message('Category','Order Cart Full')
            popItem4.value = 0
      popItem4_qty.observe(on_change)

      popItem4_Line1 = widgets.HBox([popItem4_label])
      popItem4_Line2 = widgets.HBox([popItem4_fName_label, popItem4_fName])
      popItem4_Line3 = widgets.HBox([popItem4_fDesc_label, popItem4_fDesc])
      popItem4_Line4 = widgets.HBox([popItem4_fPrice_label,popItem4_fPrice])
      popItem4_Line5 = widgets.HBox([popItem4_fSV_label,popItem4_fSV])
      popItem4_Line6 = widgets.HBox([popItem4_qty])
      popItem4_layout = widgets.Layout(flex_flow='col', border='1px solid grey')
      popItem4_VBox = widgets.VBox([popItem4_Line1,popItem4_Line2,popItem4_Line3,popItem4_Line4,popItem4_Line5,popItem4_Line6], layout = popItem4_layout)
      pop_grid[23:30,:] = popItem4_VBox

    if len(tup)>4:
      popItem5 = tup[4][0]
      popItem5_label = widgets.Label('Number 5')
      popItem5_fName_label = widgets.Label('Food Name: ')
      popItem5_fDesc_label = widgets.Label('Description: ')
      popItem5_fPrice_label = widgets.Label('Food Price: ')
      popItem5_fSV_label = widgets.Label('Sales Volume: ')

      popItem5_fName = widgets.Label(allFoodItemsDt[popItem5][1])
      popItem5_fDesc = widgets.Label(allFoodItemsDt[popItem5][2])
      popItem5_fPrice = widgets.Label(str(allFoodItemsDt[popItem5][4]))
      popItem5_fSV = widgets.Label(str(allFoodItemsDt[popItem5][8]))
      #update the dropdown value
      popItem5_ordQty = 0
      if popItem5 in getOrderCart():
        popItem5_ordQty=getOrderCart()[popItem5]

      popItem5_qty = widgets.Dropdown(options=quantity,value = popItem5_ordQty)
      def on_change(change):
        if change['type']=='change' and change['name']=='value':
          newQty=change['new']
          if not update_cart(popItem5,newQty):
            show_message('Category','Order Cart Full')
            popItem5.value = 0
      popItem5_qty.observe(on_change)

      popItem5_Line1 = widgets.HBox([popItem5_label])
      popItem5_Line2 = widgets.HBox([popItem5_fName_label, popItem5_fName])
      popItem5_Line3 = widgets.HBox([popItem5_fDesc_label, popItem5_fDesc])
      popItem5_Line4 = widgets.HBox([popItem5_fPrice_label,popItem5_fPrice])
      popItem5_Line5 = widgets.HBox([popItem5_fSV_label,popItem5_fSV])
      popItem5_Line6 = widgets.HBox([popItem5_qty])
      popItem5_layout = widgets.Layout(flex_flow='col', border='1px solid grey')
      popItem5_VBox = widgets.VBox([popItem5_Line1,popItem5_Line2,popItem5_Line3,popItem5_Line4,popItem5_Line5,popItem5_Line6], layout = popItem5_layout)
      pop_grid[30:37,:] = popItem5_VBox


  displayPopItems(get_top_5_popular_items())


  display(pop_grid)


showPage('Function Selection')